In [17]:
import numpy as np
import pprint
import sys
if "../" not in sys.path:
  sys.path.append("../") 
from lib.envs.gridworld import GridworldEnv

In [18]:
pp = pprint.PrettyPrinter(indent=2)
env = GridworldEnv()

In [19]:
# Taken from Policy Evaluation Exercise!

def policy_eval(policy, env, discount_factor=1.0, theta=0.00001):
    """
    Evaluate a policy given an environment and a full description of the environment's dynamics.
    
    Args:
        policy: [S, A] shaped matrix representing the policy.
        env: OpenAI env. env.P represents the transition probabilities of the environment.
            env.P[s][a] is a list of transition tuples (prob, next_state, reward, done).
            env.nS is a number of states in the environment. 
            env.nA is a number of actions in the environment.
        theta: We stop evaluation once our value function change is less than theta for all states.
        discount_factor: Gamma discount factor.
    
    Returns:
        Vector of length env.nS representing the value function.
    """
    # Start with a random (all 0) value function
    V = np.zeros(env.nS)

    num_updates = 0
    while True:
        num_updates += 1
        
        delta = 0
        # For each state, perform a "full backup"
        for s in range(env.nS):
            v = 0
            # Look at the possible next actions
            for a, action_prob in enumerate(policy[s]):
                # For each action, look at the possible next states...
                for  prob, next_state, reward, done in env.P[s][a]:
                    # Calculate the expected value
                    v += action_prob * prob * (reward + discount_factor * V[next_state])
            # How much our value function changed (across any states)
            delta = max(delta, np.abs(v - V[s]))
            V[s] = v
        # Stop evaluating once our value function change is below a threshold
        if num_updates % 100 == 0 :
            print(' num of updates = ', num_updates)
            print('latest V = ', V)
        
        
        if delta < theta:
            break
    return np.array(V)

In [37]:
def policy_improvement(env, policy_eval_fn=policy_eval, discount_factor=1.0):
    """
    Policy Improvement Algorithm. Iteratively evaluates and improves a policy
    until an optimal policy is found.
    
    Args:
        env: The OpenAI envrionment.
        policy_eval_fn: Policy Evaluation function that takes 3 arguments:
            policy, env, discount_factor.
        discount_factor: gamma discount factor.
        
    Returns:
        A tuple (policy, V). 
        policy is the optimal policy, a matrix of shape [S, A] where each state s
        contains a valid probability distribution over actions.
        V is the value function for the optimal policy.
        
    """
    # Start with a random policy
    policy = np.ones([env.nS, env.nA]) / env.nA
    
    while True:
        # Implement this!

        # policy valuation step:
        print('valuating')
        value_of_current_policy = policy_eval_fn(policy, env, discount_factor = discount_factor )
        
        print('updating policy')
        # policy improvement step:
        policy_stable = True
        for s in range(env.nS):
            print('updating s = ', s)
            chosen_a = np.argmax(policy[s])
            print('chosen_a = ', chosen_a)
            # update the policy given the value of latest_policy:
            
            best_a = np.argmax(
                
                [
                    sum([ 
                        prob*(reward + discount_factor*value_of_current_policy[next_s])
                for (prob, next_s, reward, _ ) in env.P[s][a]])
                for a in range(env.nA)
                ]
            )

            print('best_a', best_a)
            # print(policy.shape)
            policy[s] = np.eye(env.nA)[best_a]

            if best_a != chosen_a:
                policy_stable = False 
        if not policy_stable:
            print('policy not stable, keep going...')
        if policy_stable:
            print('policy stable! ')
            break
       # else:
            #print('not equal', policy, 'vs', old_policy)
    
    return policy, value_of_current_policy# np.zeros(env.nS)

In [38]:
policy, v = policy_improvement(env)
print("Policy Probability Distribution:")
print(policy)
print("")

print("Reshaped Grid Policy (0=up, 1=right, 2=down, 3=left):")
print(np.reshape(np.argmax(policy, axis=1), env.shape))
print("")

print("Value Function:")
print(v)
print("")

print("Reshaped Grid Value Function:")
print(v.reshape(env.shape))
print("")



valuating
 num of updates =  100
latest V =  [  0.         -13.99765839 -19.99663362 -21.99629468 -13.99765839
 -17.99712654 -19.99688008 -19.99691576 -19.99663362 -19.99688008
 -17.99736736 -13.99803444 -21.99629468 -19.99691576 -13.99803444
   0.        ]
updating policy
updating s =  0
chosen_a =  0
best_a 0
updating s =  1
chosen_a =  0
best_a 3
updating s =  2
chosen_a =  0
best_a 3
updating s =  3
chosen_a =  0
best_a 3
updating s =  4
chosen_a =  0
best_a 0
updating s =  5
chosen_a =  0
best_a 0
updating s =  6
chosen_a =  0
best_a 3
updating s =  7
chosen_a =  0
best_a 2
updating s =  8
chosen_a =  0
best_a 0
updating s =  9
chosen_a =  0
best_a 0
updating s =  10
chosen_a =  0
best_a 1
updating s =  11
chosen_a =  0
best_a 2
updating s =  12
chosen_a =  0
best_a 0
updating s =  13
chosen_a =  0
best_a 1
updating s =  14
chosen_a =  0
best_a 1
updating s =  15
chosen_a =  0
best_a 0
policy not stable, keep going...
valuating
updating policy
updating s =  0
chosen_a =  0
best_a 

In [39]:
# Test the value function
expected_v = np.array([ 0, -1, -2, -3, -1, -2, -3, -2, -2, -3, -2, -1, -3, -2, -1,  0])
np.testing.assert_array_almost_equal(v, expected_v, decimal=2)